In [1]:
!pip install langchain
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.0/396.0 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 2.2 MB/s eta 0:00:00


In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install pypdf
!pip install "unstructured[local-inference]"
!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
!pip install layoutparser[layoutmodels,tesseract]

In [33]:
!pip install colab-env --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3829 sha256=d64d7684f5424fe43e1d288aaefd2417a99f48e534493c225dee110e21dc6780
  Stored in directory: /root/.cache/pip/wheels/e1/25/d4/95a3d7b269bddc5c6b91e2995e446c01cb03e6472fc5c61507
Successfully built colab-env
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0


Google Driveにvars.envを作成し、OPENAI_API_KEYを設定する

In [34]:
import colab_env

Mounted at /content/gdrive


In [36]:
from langchain.llms import OpenAI

In [37]:
llm = OpenAI(temperature=0.9)
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Rainbow Toes Socks.


# ダイキン工業の2023年2月の報告書をダウンロードして配置する
https://disclosure2.edinet-fsa.go.jp/WEEE0030.aspx?bXVsPSVFMyU4MyU4MCVFMyU4MiVBNCVFMyU4MiVBRCVFMyU4MyVCMyZjdGY9b2ZmJmZscz1vbiZscHI9b2ZmJnJwcj1vZmYmb3RoPW9mZiZ5ZXI9Jm1vbj0mcGZzPTYmc2VyPTEmcGFnPTEmc29yPTI=

PDFがそのままだと文字化けして読めなかったので、いったんコピペしてtextにした
ToDo
XMLやPDFからテキスト、テーブルの構造化データを読める形に変換する

In [38]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [39]:
from langchain.document_loaders import UnstructuredFileLoader

# loader = UnstructuredFileLoader("/content/test.txt", mode="elements")
loader = UnstructuredFileLoader("/content/test.txt")
docs = loader.load()
docs[0].page_content[:400]

'EDINET提出書類\n\nダイキン工業株式会社(E01570)\n\n四半期報告書\n\n【表紙】\n\n【提出書類】四半期報告書\n\n【根拠条文】金融商品取引法第24条の４の７第１項\n\n【提出先】関東財務局長\n\n【提出日】2023年２月８日\n\n【四半期会計期間】第120期第３四半期(自\n\n【会社名】ダイキン工業株式会社\n\n【英訳名】DAIKIN INDUSTRIES,LTD.\n\n【代表者の役職氏名】取締役社長\n\n【本店の所在の場所】大阪市北区梅田一丁目13番１号大阪梅田ツインタワーズ・サウス\n\n十\n\n河\n\n2022年10月１日\n\n政\n\n至\n\n2022年12月31日)\n\n則\n\n(2022年11月24日より本店所在地\n\n大阪市北区中崎西二丁目４番12号梅田セン\n\nタービルが上記のように移転しております。)\n\n【電話番号】大阪(06)6147-6864\n\n【事務連絡者氏名】経理財務本部経理グループ長\n\n【最寄'

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    # separator = "．",  # セパレータ # ToDo別のやつのほうがいい
    chunk_size = 256,  # チャンクの文字数
    chunk_overlap = 0,  # チャンクオーバーラップの文字数
)

In [40]:
with open('/content/test.txt', 'rt') as f:
    text = f.read()
    texts = text_splitter.split_text(text)

In [41]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA

db = Chroma.from_texts(texts, embeddings)
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=db)

query = "今季の売上を教えてください"
qa.run(query)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:time to pre process our knn query: 4.291534423828125e-06
DEBUG:Chroma:time to run knn query: 0.0005064010620117188


ValidationError: ignored

In [6]:
query = "今季の売上を教えてください"
index.query(query)

DEBUG:Chroma:time to pre process our knn query: 3.0994415283203125e-06
DEBUG:Chroma:time to run knn query: 0.00022602081298828125


InvalidRequestError: ignored